## Script to compute SB in different frequencies
The goal of this script it's to provide the way to compute SB with current qubicsoft

In [ ]:
import os
import qubic
import healpy as hp
import numpy as np
import pylab as plt
from matplotlib.pyplot import *
from qubic import QubicInstrument as QI
from qubic import QubicMultibandInstrument as QMI

import sys
from pysimulators import FitsArray

%matplotlib inline

def select_det(q,id):
    id=[id,]
    detector_i = q.detector[id]
    q.detector = detector_i
    return(q)

def lamb2nu(var, inverse = False):
    """[lamb] = mm
    c = lamb * nu
    return in GHz
    
    if inverse: give nu[GHz] and return lamda[mm] 
    """
    c = 3e11 # mm/sec
    if inverse:
        return c/var/1e9
    else:
        return c/var/1e9

def primary_beam(theta, Lambda=2.):
    """Gaussian primary beam"""
    #sig = np.radians(13./2)
    sig = 13./2
    B_prim = np.exp(-0.5 * (theta ** 2 / (  Lambda * sig) ** 2))
    return B_prim

name='test_scan_source'
resultDir='%s'%name


In [ ]:
alaImager=False
component=0
sel_det=True #If you want to select one detector

# INSTRUMENT
d = qubic.qubicdict.qubicDict()
d.read_from_file('global_source_oneDet.dict')
d['nside'] = 512
d['RA'] = 0
d['DEC'] = 0
d['MultiBand']=False
d['nf_sub'] =2
d['beam_shape']='multi_freq'
print(d['MultiBand'], d['beam_shape'])

***
Check location of the TES

q2 = QI(d)
arrdet = [q2.detector[231],q2.detector[231+248], q2.detector[231+496], q2.detector[231+248+496]]
plot(arrdet[0].vertex[0][2][0],arrdet[0].vertex[0][2][1], 'r*')
plot(arrdet[1].vertex[0][2][0],arrdet[1].vertex[0][2][1], 'r*')
plot(arrdet[2].vertex[0][2][0],arrdet[2].vertex[0][2][1], 'r*')
plot(arrdet[3].vertex[0][2][0],arrdet[3].vertex[0][2][1], 'r*')
q2.detector.plot()
iarrdet = [231,479,727,975]

In [ ]:
q = QI(d)
manyfreq = [140e9,158e9]
size=200
maps = np.empty((len(manyfreq), size,size))
ndet = 12#iarrdet

if sel_det:
    q = select_det(q[:],ndet)


for i,ifreq in enumerate(manyfreq):

    pos = q.detector.center
    area = q.detector.area
    q.filter.nu = ifreq
    nu = ifreq#q.filter.nu
    print(nu)
    bw = q.filter.bandwidth
    horns = q.horn
    pbeam = q.primary_beam
    sbeam = q.secondary_beam
    btype = q.synthbeam.dtype    

    s = qubic.QubicScene(d)
    
    sb = q.get_synthbeam(s)
    print(sb.shape)
    xr=0.95*np.max(sb)

    sat = np.where(sb>xr)
    sb2=np.copy(sb)
    sb2[sat] = xr
    #sbr = sb2.ravel()[::-1]
    #sb2 = sbr.reshape(sb.shape)
    maps[i,:,:] = hp.gnomview(sb2[0]/np.max(sb2[0]), rot=[0,90], xsize=size, ysize=size, reso=10, 
                              title='{}'.format(str(ifreq)[:3]), sub=(2,3,i+1), min=0, max=1,
                              cmap='viridis',return_projected_map=True)


d1=d.copy()
d1['config'] = 'FI'
d1['hornarray']='CalQubic_HornArray_CC.fits'
d1['detarray']='CalQubic_DetArray_CC.fits'
qnew = qubic.QubicInstrument(d1)
qnew.horn.plot()
d['hornarray']

In [ ]:
full = np.sum([maps[0],maps[-1]], axis= 0)

print(full.shape)
plt.imshow(full)

In [ ]:
#zoom in to see how secondary peak closer 
figure(figsize=(12,10))

for i in range(len(maps)):
    subplot(2,3,i+1)
    xlim(30,100)
    ylim(75,125)
    imshow(maps[i,:,:])

***

Do the same but with one horn open

In [ ]:
q_one = QI(d)
maps_one = np.empty((len(manyfreq), size,size))
ndet = 12#iarrdet
q_one.horn.open[~20] = False 
q_one.horn.open = ~q_one.horn.open
if sel_det:
    q = select_det(q_one,ndet)

for i,ifreq in enumerate(manyfreq):

    q_one.filter.nu = ifreq
    nu = ifreq
    
    print(nu)
    bw = q_one.filter.bandwidth
    pb = q_one.get_synthbeam(s)

    s = qubic.QubicScene(d)
    
    print(pb.shape)
    xr=0.95*np.max(pb)

    sat = np.where(pb>xr)
    pb2=np.copy(pb)
    pb2[sat] = xr
    maps_one[i,:,:] = hp.gnomview(pb2[0]/np.max(pb2[0]), rot=[0,90], xsize=size, ysize=size, reso=10, 
                              title='{}'.format(str(ifreq)[:3]), sub=(2,3,i+1), min=0, max=1,
                              cmap='viridis',return_projected_map=True)

    


***

Cut the beams

In [ ]:
maxx = []
maxy = []
newmap = []
newmap_one =[]
half = int(size/2)
reso=1.5

maxx_0, maxy_0 = np.unravel_index(np.argmax(maps[0]), dims=(size,size))
newmap.append(np.roll(np.roll(maps[0], half-maxx_0, axis=0), half-maxy_0, axis=1))
maxx_1, maxy_1 = np.unravel_index(np.argmax(maps[1]), dims=(size,size))
newmap.append(np.roll(np.roll(maps[1], half-maxx_1, axis=0), half-maxy_1, axis=1))

maxx_0_one, maxy_0_one = np.unravel_index(np.argmax(maps_one[0]), dims=(size,size))
newmap_one.append(np.roll(np.roll(maps_one[0], half-maxx_0_one, axis=0), half-maxy_0_one, axis=1))
maxx_1_one, maxy_1_one = np.unravel_index(np.argmax(maps_one[1]), dims=(size,size))
newmap_one.append(np.roll(np.roll(maps_one[1], half-maxx_1_one, axis=0), half-maxy_1_one, axis=1))

print(maxx_0, maxx_1)

dd=True
xd = np.linspace(-100,100,200)*reso/60
if dd: 
    xd = np.linspace(-100,100,200)*reso/60*np.sqrt(2)
    x=xd
    
plt.figure(figsize=(10,6))
plt.ylim(0,1.0)
plt.ylabel('Synthesized beam', fontsize=18)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.xlabel(r'$\theta$[deg]', fontsize=18)

plt.plot(x,newmap[0][100,:], 'm-', label = '{} GHz'.format(int(manyfreq[0]/1e9)), 
         linewidth=3, alpha = 0.8)
plt.plot(x,newmap[1][100,:], 'c-', label = '{} GHz'.format(int(manyfreq[1]/1e9)), 
         linewidth=3, alpha=0.8)
plt.plot(x,newmap_one[0][100,:], 'k-', label = 'One horn {} GHz'.format(int(manyfreq[0]/1e9)), 
         linewidth=3, alpha=0.4)
plt.plot(x,newmap_one[1][100,:], 'k-', label = 'One horn {} GHz'.format(int(manyfreq[1]/1e9)), 
         linewidth=3, alpha=0.8)


plt.legend(loc='best', fontsize=16)



***
Sum all maps into one

In [ ]:
full_ = np.sum(maps, axis= 0)
print(full_.shape)
newfull=[]
maxx_f, maxy_f = np.unravel_index(np.argmax(full), dims=(size,size))
newfull.append(np.roll(np.roll(full_, half-maxx_f, axis=0), half-maxy_f, axis=1))

dd=True
xd = np.linspace(-100,100,200)*reso/60
if dd: 
    xd = np.linspace(-100,100,200)*reso/60*np.sqrt(2)
    x=xd
plt.figure(figsize=(10,6))
plt.subplot(121)
plt.imshow(newfull[0])
plt.subplot(122)
plt.ylabel('Synthesized beam', fontsize=18)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.xlabel(r'$\theta$[deg]', fontsize=18)
plt.plot(x,np.diag(newfull[0]), 'm-',  linewidth=3)


***
### Now MultibandInstrument

In [ ]:
size=200
half=size/2

d['beam_shape'] = 'multi_freq'
def select_mband_det(q,id):
    id=[id]
    detector_i = q.detector[id]
    q.detector = detector_i
    return(q.detector)

# MULTIBAND INSTRUMENT
dmb = qubic.qubicdict.qubicDict()
dmb.read_from_file('global_source_oneDet.dict')
dmb['nside'] = 512
dmb['RA'] = 0
dmb['DEC'] = 0
d['nf_sub'] =2

sel_det=True
ndet = 12
if sel_det:
    qmb = qubic.QubicMultibandInstrument(d)
    for i,qi in enumerate(qmb):
        qmb[i].detector = select_mband_det(qi,ndet)

numb = np.zeros((len(qmb), ) )
sbeammb = np.zeros((len(qmb), ) )

posmb = qmb[0].detector.center
areamb = qmb[0].detector.area
for i,iq in enumerate(qmb):
    numb[i] = iq.filter.nu

sbeammb = [i.secondary_beam for i in qmb]

mapsmb = np.zeros((len(manyfreq), size,size))
sec_beam= np.zeros((len(manyfreq), size,size))
s = qubic.QubicScene(dmb)    
for j,qi in enumerate(qmb):
    sbmb = qmb[j].get_synthbeam(s)[0]
    
    xr=0.95*np.max(sbmb)
    sat = np.where(sbmb>xr)
    sb2mb=np.copy(sbmb)
    sb2mb[sat] = xr
    plt.figure(figsize=(10,10))
    mapsmb[j,:,:] = hp.gnomview(sb2mb/np.max(sb2mb), rot=[0,90], xsize=size, ysize=size, reso=10, 
                              title='{}'.format(str(numb[j])[:3]), sub=(4,3,j+1), min=0, max=1,
                              cmap='viridis',return_projected_map=True)
    plt.figure(figsize=(10,10))
    sec_beam[j,:,:] = hp.gnomview(sbeammb[j].healpix(d['nside'])/np.max(sbeammb[j].healpix(d['nside'])), 
                                  rot=[0,-90], xsize=size, ysize=size, reso=10, 
                              title='{}'.format(str(numb[j])[:3]), sub=(4,3,j+1), min=0, max=1,
                              cmap='viridis',return_projected_map=True)

In [ ]:
maxx = []
maxy = []
newmap = []
newmap_mb = []
half = int(size/2)
reso=10

maxx_0, maxy_0 = np.unravel_index(np.argmax(mapsmb[0]), dims=(size,size))
newmap.append(np.diag(np.roll(np.roll(mapsmb[0], half-maxx_0, axis=0), half-maxy_0, axis=1)))
maxx_1, maxy_1 = np.unravel_index(np.argmax(mapsmb[1]), dims=(size,size))
newmap.append(np.diag(np.roll(np.roll(mapsmb[1], half-maxx_1, axis=0), half-maxy_1, axis=1)))

maxx_0_mb, maxy_0_mb = np.unravel_index(np.argmax(sec_beam[0]), dims=(size,size))
newmap_mb.append(np.diag(np.roll(np.roll(sec_beam[0], half-maxx_0_mb, axis=0), half-maxy_0_mb, axis=1)))
maxx_1_mb, maxy_1_mb = np.unravel_index(np.argmax(sec_beam[1]), dims=(size,size))
newmap_mb.append(np.diag(np.roll(np.roll(sec_beam[1], half-maxx_1_mb, axis=0), half-maxy_1_mb, axis=1)))

wlen = lamb2nu(int(numb[0]/1e9), inverse = True)
wlen2 = lamb2nu(int(numb[1]/1e9), inverse = True)

dd=True
x = np.linspace(-100,100,200)*reso/60 #deg

if dd: 
    xd = np.linspace(-100,100,200)*reso/60*np.sqrt(2)
    x=xd
    
plt.figure(figsize=(10,6))
plt.ylim(0,1.0)
plt.ylabel('Synthesized beam', fontsize=18)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.xlabel(r'$\theta$[deg]', fontsize=18)

plt.plot(x,newmap[0], 'm-', label = '{} GHz'.format(int(numb[0]/1e9)), linewidth=3, alpha = 0.8)
#plt.plot(x,primary_beam(x, Lambda=wlen )**2, 'm--', label='{:3.2}cm ({})'.format(wlen,int(numb[0]/1e9)))
#plt.plot(x,primary_beam(x, Lambda=wlen2 )**2, 'c--', label='{:3.2}cm ({})'.format(wlen,int(numb[1]/1e9)))
plt.plot(x,newmap[1], 'c-', label = '{} GHz'.format(int(numb[1]/1e9)), linewidth=3, alpha=0.8)
plt.plot(x,newmap_mb[0], 'm-', label = '{} GHz'.format(int(numb[1]/1e9)), linewidth=3, alpha=0.3)
plt.plot(x,newmap_mb[1], 'c-', label = '{} GHz'.format(int(numb[1]/1e9)), linewidth=3, alpha=0.3)

plt.axhline(0.5, c='k', ls='-.')
plt.axvline(13, c='k', ls='-.',alpha=0.2)
plt.axvline(-13, c='k', ls='-.',alpha=0.2)
plt.legend()



In [ ]:
hp.gnomview(sbeammb[0].healpix(256), rot=[0,-90], sub=(1,2,1), reso=15)
hp.gnomview(sbeammb[-1].healpix(256), rot=[0,-90], sub=(1,2,2), reso=15)